_A problem can be defined formally by five components_:  
* **Initial State**: the state that the agent starts in.
* **Actions**: set of all possible actions the agent may take, action space denoted as 'a'.
* **Transition Model**: description of what each action does, specified by function 'RESULTS(s,a)' which returns the state that results from doing action a in state s.
* **Goal Test**: determines whether a given state is a goal state.
* **Path Cost**: function that assigns a numeric cost to each path.

In [1]:
'''
Environment object takes parameters name (string)
'''
class Environment:
    def __init__(self):
        # Initialize 2x2 grid environment as 2D matrix
        w, h = 2, 2
        self.grid = [[0 for x in range(w)] for y in range(h)] 

    def set_position_state(self, x, y, dirt_bool):
        # Given x,y coords of the grid, sets the value of grid to 1 for dirt present, 0 if no dirt present (dirt_bool)
        self.grid[x][y] = dirt_bool
    
    def get_position_state(self,x,y):
        # Current state of the environment position given x,y: For each position 1 = dirt present , 0 = No dirt present
        return self.grid[x][y]

In [2]:
'''
Agent object takes parameters grid (Environment object), x, y (initial state coordinates, int 0 or 1)
'''
class Agent:
    def __init__(self, environment, x, y):
        
        # Actual environment/world
        self.environment = environment
        
        
        # Agent current status: idle (waiting start), perceiving, thinking, acting, complete (finished)
        self.current_status = "idle"
        
        # Agent current position: x,y coordinates of position in 2x2 grid environment
        self.x = x
        self.y = y
        
        # Goal reached boolean - if goal_test() determines state is goal state, updates goal_bool = 1
        self.goal_bool = 0
        
    def get_current_position(self):
        return self.x,self.y
    
    
    # Actions - Suck, Move North, Move East, Move South, Move West
    def suck(self):
        # Removes dirt from present location
        current_status = "acting"
        if (self.environment.get_position_state(self.x,self.y) != 0):
            self.environment.set_position_state(self.x,self.y,0)
            print("Sucked up dirt at position: (",self.x, ",", self.y, ")")
        
    def move_north(self):
        # Moves agent up 1 location
        current_status = "acting"
        
        if(self.y != 1):
            self.y = 1
            
        print("Moved north.")
        
    def move_east(self):
        # Moves agent right 1 location
        current_status = "acting"
        
        if(self.x != 1):
            self.x = 1
        
        print("Moved east.")
        
    def move_south(self):
        # Moves agent down 1 location
        current_status = "acting"
        
        if(self.y == 1):
            self.y = 0
        
        print("Moved south.")
        
    def move_west(self):
        # Moves agent left 1 location
        current_status = "acting"
        
        if(self.x == 1):
            self.x = 0
        
        print("Moved west.")
            
    def search(self):
        # Offline depth first limited state space search.
        current_status = "thinking"
        print("Searching.")
        
        
    def complete(self):
        current_status = "complete"
        
        if(self.goal_bool == 1):
            print("Complete: Solution found!")
        else:
            print("Complete: No solution found.")

In [3]:
def main():
    # Instantiate a new environment
    environment = Environment()
    
    # Testing environment functions
    # Set the environment state: for each position either 1 = dirt or 0 = no dirt.
    environment.set_position_state(0,0,1)
    environment.set_position_state(0,1,0)
    environment.set_position_state(1,1,1)
    environment.set_position_state(1,0,0)
    
    # Print the environment state of each position.
    print(environment.get_position_state(0,0))
    print(environment.get_position_state(0,1))
    print(environment.get_position_state(1,1))
    print(environment.get_position_state(1,0))
  
    
    # Instantiate a new agent
    agent = Agent(environment, 0, 0)
    
    
    # Testing agent functions
    agent.suck()
    print("Agent current position: ", agent.get_current_position())
    agent.move_north()
    print("Agent current position: ", agent.get_current_position())
    agent.move_east()
    print("Agent current position: ", agent.get_current_position())
    agent.move_south()
    print("Agent current position: ", agent.get_current_position())
    agent.move_west()
    print("Agent current position: ", agent.get_current_position())
    agent.search()
   
    # Define the initial environment state (which cells contain dirt / no dirt)
    
    # Define the agent's starting point (which position the agent will begin search in the grid)
    
    
    # Define the goal state: all 4 locations contain no dirt, agent location irrelevant when reached.

if __name__ == "__main__":
  main()

1
0
1
0
Sucked up dirt at position: ( 0 , 0 )
Agent current position:  (0, 0)
Moved north.
Agent current position:  (0, 1)
Moved east.
Agent current position:  (1, 1)
Moved south.
Agent current position:  (1, 0)
Moved west.
Agent current position:  (0, 0)
Searching.


In [4]:
class State:
    def __init__(self, name, environment_state, agent_state, parent=None):
        self.name = name
        self.environment_state = environment_state
        self.agent_state = agent_state
        self.parent = parent
        self.depth = ''
        if (self.parent == None):
            self.depth = 0
        else:
            self.depth = self.parent.depth + 1
        
    def isGoalState(self):
        loc_A = self.environment_state.get_position_state(0,0)
        loc_B = self.environment_state.get_position_state(0,1)
        loc_C = self.environment_state.get_position_state(1,1)
        loc_D = self.environment_state.get_position_state(1,0)
        
        if(loc_A != 1 and loc_B != 1 and loc_C != 1 and loc_D != 1):
            return True
        else:
            return False
            
    def path(self):
        "Create a list of nodes from the root to this node."
        x, result = self, [self]
        while x.parent:
            result.append(x.parent)
            x = x.parent
        return result

    def expand(self):
        actions = ['suck', 'move north', 'move east', 'move south', 'move west']
        successor_nodes = []
        
        # If this is level 1 - that is the first level from root/init state then no actions made to states.
        if(self.parent == None):
            
            #print("root debug")
            
            for action in actions:
                    successor_nodes.append(State(action,self.environment_state, self.agent_state, self))
            return successor_nodes
        
        # Parent state/decision/node was 'suck' action, update the virtual environment and expand
        if(self.name == 'suck'):
            
            #print("suck debug")
            
            # Suck up dirt from virtual copy at current agent position if any is present.
            self.environment_state.set_position_state(self.agent_state.x,self.agent_state.y,0)
            for action in actions:
                successor_nodes.append(State(action,self.environment_state, self.agent_state, self))
            return successor_nodes
        
        # Parent state/decision/node was 'move north' action, update the virtual environment and expand
        if(self.name == 'move north'):
            
            
            #print("north debug")
            
            # Move north in virtual copy, then expand nodes
            if (self.agent_state.y != 1):
                self.agent_state.y = 1
                
            for action in actions:
                successor_nodes.append(State(action,self.environment_state, self.agent_state, self))
            return successor_nodes
            
        # Parent state/decision/node was 'move east' action, update the virtual environment and expand
        if(self.name == 'move east'):
            
            #print("east debug")
            
            # Move east in virtual copy, then expand nodes
            if (self.agent_state.x != 1):
                self.agent_state.x = 1
                
            for action in actions:
                successor_nodes.append(State(action,self.environment_state, self.agent_state, self))
            return successor_nodes
            
        # Parent state/decision/node was 'move south' action, update the virtual environment and expand
        if(self.name == 'move south'):
            
            
            #print("south debug")
            
            # Move south in virtual copy, then expand nodes
            if (self.agent_state.y == 1):
                self.agent_state.y = 0 
                
            for action in actions:
                successor_nodes.append(State(action,self.environment_state, self.agent_state, self))
            return successor_nodes
            
        # Parent state/decision/node was 'move west' action, update the virtual environment and expand
        if(self.name == 'move west'):
            
            #print("west debug")
            #print(self.agent_state.x)
            
            # Move west in virtual copy, then expand nodes
            if (self.agent_state.x == 1):
                self.agent_state.x = 0 
                #print(self.agent_state.x)
            for action in actions:
                successor_nodes.append(State(action,self.environment_state, self.agent_state, self))
            return successor_nodes

In [5]:
''' Testing the environment, agent, and state functions'''
# Instantiate a new 2x2 environment grid
world = Environment()

# Instantiate a new Agent given: the environment, (x,y) coordinate starting position of agent.
agent = Agent(world, 1, 0)

# Instantiate a new State given: name, environment, and agent.
state = State('root',world,agent)

# Print out the state of each position in the environment: 1 = dirt present, 0 = no dirt
print(world.get_position_state(0,0))
print(world.get_position_state(0,1))
print(world.get_position_state(1,0))
print(world.get_position_state(1,1))

print()

# Test if this state is a goal state: no dirt present in any of the four positions.
print(state.isGoalState())

print()

# Place dirt in positions: (0,0) , (0,1)
world.set_position_state(0,0,1)
world.set_position_state(0,1,1)
world.set_position_state(1,0,0)
world.set_position_state(1,1,0)

# Test if goal state again, should return False.
print(state.isGoalState())

print()

# Test state.expand() - Expands a state node and returns a list of successor states/nodes
children = state.expand()

print()

# Count the number of successor states/nodes - Should return 5: suck, move north, move east, move south, move west.
print(len(children))

print()

# Check if all the expanded nodes have proper parent node reference from -- > root
for child in children:
    print(child.parent.name)

print()

# Check if all child nodes have proper names associated with actions in proper order
for child in children:
    print(child.name)

print()

# Check if all child nodes have proper depth associated - Should return depth = 1 for all 5 children states
for child in children:
    print(child.depth)

print()

# Test child state/node for name, and parent node name: Should yield parent = root, name = move west
child = children.pop()
print('parent: ', child.parent.name)
print('child name: ', child.name)

print()

# Expand child node 'move west', pop last child 'move west' again, verify name and parent node name's match again
round2 = child.expand()
child = round2.pop()
print('parent: ', child.parent.name)
print('child name: ', child.name)

print()

# Expand child node 'move west', pop 2nd last child 'move south', verify parent = 'move west' and name = 'move south'
round3 = child.expand()
round3.pop()
child = round3.pop()
print('parent: ', child.parent.name)
print('child name: ', child.name)

print()

# Expand child node 'move south', pop 3rd last child 'move east', verify parent = 'move south' and name = 'move east'
round4 = child.expand()
round4.pop()
round4.pop()
child = round4.pop()
print('parent: ', child.parent.name)
print('child name: ', child.name)

print()

# Expand child node 'move east', pop 4th last child 'move north', verify parent = 'move east' and name = 'move north'
round5 = child.expand()
round5.pop()
round5.pop()
round5.pop()
child = round5.pop()
print('parent: ', child.parent.name)
print('child name: ', child.name)

print()

# Expand child node 'move north', pop first child 'suck', verify parent = 'move north' and name = 'suck'
round6 = child.expand()
round6.pop()
round6.pop()
round6.pop()
round6.pop()
child = round6.pop()
print('parent: ', child.parent.name)
print('child name: ', child.name)

print()

0
0
0
0

True

False


5

root
root
root
root
root

suck
move north
move east
move south
move west

1
1
1
1
1

parent:  root
child name:  move west

parent:  move west
child name:  move west

parent:  move west
child name:  move south

parent:  move south
child name:  move east

parent:  move east
child name:  move north

parent:  move north
child name:  suck



In [6]:
def depth_limited_search(initial_state,limit=10):
    """[Figure 3.17]"""
    def recursive_dls(state,limit):
        print(state.name)
        if (state.isGoalState()):
            return state
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in state.expand():
                result = recursive_dls(child, limit - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(initial_state, limit)

In [7]:
''' Setup a new environment, agent, and initial state for search space traversal '''

# New 2x2 grid environment
world = Environment()

# New agent for world environment, agent starting position = (0,0)
agent = Agent(world, 0, 0)

# Dirt placed at: (0,0) and (0,1)
world.set_position_state(0,0,1)
world.set_position_state(0,1,1)
world.set_position_state(1,0,0)
world.set_position_state(1,1,0)

# Instantiate the initial state/root node containing the environment dirt positions (if any), and agent position.
root = State('root',world,agent)

In [8]:
# Perform a recursive depth limited search with limit depth = 10, on the generated state search space 
goal = depth_limited_search(root, 10)

root
suck
suck
suck
suck
suck
suck
suck
suck
suck
suck
move north
move east
move south
move west
move north
suck
move north
move east
move south
move west
move east
suck
move north
move east
move south
move west
move south
suck
move north
move east
move south
move west
move west
suck
move north
move east
move south
move west
move north
suck
suck


In [26]:
solution = goal.path()

In [27]:
# Output the solution state path found by the agent.
count = len(solution)
while (count >0):
    print(solution.pop().name)
    count -= 1

root
suck
suck
suck
suck
suck
suck
suck
move north
suck
suck
